### This code taken and/or adapted from [QuantPY](https://quantpy.com.au/)

In [1]:
# import os
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr

# import scipy as sci
from scipy import stats
import scipy.optimize as sco

import yfinance as yf

import datetime as dt
from datetime import date
from datetime import timedelta

import plotly.graph_objects as go

In [2]:
# Specify date range for data
# Set start and end dates of 5 years "5Y" back from your current date
# Define the time delta (time-frame) to start with
time_delta = 10

end_date = pd.to_datetime('today')
# 10 years
start_date = end_date - np.timedelta64(time_delta, 'Y')
start, end = start_date, end_date
# Verify date range
print(f"Start date: {start} and End date: {end}")

Start date: 2012-10-24 01:08:00.574652 and End date: 2022-10-24 11:20:00.574652


In [3]:
# Specify portfolio sectors - basket of market index and assets
blend_list = ['IBM', 'TSLA','TSM', 'NVDA', 'DVN', 'OXY','CNI', 'NOC','BHF', 'AIG']
tech_list = ['AAPL', 'IBM', 'TSLA', 'GOOGL', 'MSFT','CSCO', 'TSM', 'NVDA']
energy_list = ['DVN', 'OXY', 'NEE', 'MRO', 'APA', 'PDC', 'CTRA']
industrials_list = ['BA', 'GE', 'CNI', 'NOC', 'LPX', 'BLDR', 'AAL', 'CPA']
financials_list = ['BHF', 'AIG', 'EQH', 'PFG', 'WBS']
market = ['^GSPC']   # adding S&P 500 Index (^GSPC) for use in beta weighting 

In [4]:
# Specify portfolio sectors - basket of market index and assets - S&P 500 Index (^GSPC) and Tech Stocks List
basket_list = market  ds.tech_list + ds.energy_list + ds.industrials_list + ds.financials_list # + blend_list

print("Basket of stocks")
print(basket_list)

Basket of stocks
['^GSPC', 'IBM', 'TSLA', 'TSM', 'NVDA', 'DVN', 'OXY', 'CNI', 'NOC', 'BHF', 'AIG']


#### Get Stock Data

In [5]:
# Import data
def getData(basket_list, start_date, end_date):
    stockData = pdr.get_data_yahoo(basket_list, start=start_date, end=end_date)
    stockData = stockData['Close']

    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

# call it
meanReturns, covMatrix = getData(basket_list, start_date, end_date)
display(meanReturns)
display(covMatrix)

Symbols
^GSPC    0.000454
IBM     -0.000018
TSLA     0.002512
TSM      0.000724
NVDA     0.001856
DVN      0.000613
OXY      0.000438
CNI      0.000480
NOC      0.000917
BHF      0.000327
AIG      0.000380
dtype: float64

Symbols,^GSPC,IBM,TSLA,TSM,NVDA,DVN,OXY,CNI,NOC,BHF,AIG
Symbols,,,,,,,,,,,
^GSPC,0.000120,0.000103,0.000169,0.000122,0.000191,0.000180,0.000157,0.000105,0.000086,0.000301,0.000148
IBM,0.000103,0.000219,0.000110,0.000100,0.000136,0.000177,0.000166,0.000096,0.000091,0.000316,0.000147
TSLA,0.000169,0.000110,0.001270,0.000222,0.000371,0.000259,0.000204,0.000128,0.000058,0.000358,0.000180
TSM,0.000122,0.000100,0.000222,0.000338,0.000274,0.000182,0.000154,0.000108,0.000058,0.000291,0.000126
NVDA,0.000191,0.000136,0.000371,0.000274,0.000756,0.000229,0.000176,0.000147,0.000099,0.000405,0.000173
DVN,0.000180,0.000177,0.000259,0.000182,0.000229,0.001018,0.000703,0.000202,0.000114,0.000669,0.000321
OXY,0.000157,0.000166,0.000204,0.000154,0.000176,0.000703,0.000890,0.000170,0.000129,0.000667,0.000301
CNI,0.000105,0.000096,0.000128,0.000108,0.000147,0.000202,0.000170,0.000198,0.000078,0.000289,0.000140
NOC,0.000086,0.000091,0.000058,0.000058,0.000099,0.000114,0.000129,0.000078,0.000220,0.000200,0.000110


In [6]:
df = pdr.get_data_yahoo(basket_list, start_date, end_date)
log_returns = np.log(df.Close / df.Close.shift(1)).dropna()
log_returns.head()

Symbols,^GSPC,IBM,TSLA,TSM,NVDA,DVN,OXY,CNI,NOC,BHF,AIG
Date,,,,,,,,,,,
2017-07-18,0.000598,0.006449,0.026769,-0.001115,0.010357,0.000319,0.003326,0.003951,-0.002155,0.000000,-0.002814
2017-07-19,0.005358,-0.042921,-0.009120,0.014942,-0.005195,0.035707,0.010732,-0.013836,0.002570,-0.015114,0.004998
2017-07-20,-0.000154,0.000881,0.014225,-0.008551,0.014432,-0.015817,-0.006095,-0.006442,-0.000566,0.000725,0.000623
2017-07-21,-0.000368,-0.003936,-0.004618,-0.006949,0.003576,-0.000938,-0.008129,-0.000366,0.002000,-0.001117,-0.000467
2017-07-24,-0.001064,-0.007439,0.042098,0.010268,-0.011668,-0.006907,-0.010382,-0.007469,-0.000113,0.001117,0.005127


### Directly calculate beta

In [9]:
def calc_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0] 
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            # stock returns are indexed by ind
            s = np_array[:,ind] 
            # Calculate covariance matrix between stock and market
            covariance = np.cov(s,m) 
            beta.append( covariance[0,1]/covariance[1,1] )
    return pd.Series(beta, df.columns[1:], name='Beta')

beta = calc_beta(log_returns)

### Define your Portfolio and make DataFrame
Calculate Beta Weighted Portfolio

In [10]:
units = np.array([100, 250, 300, 400, 200])
ASXprices = df.Close[-1:].values.tolist()[0]
price = np.array([round(price,2) for price in ASXprices[1:]])
value = [unit*pr for unit, pr in zip(units, price)]
weight = [round(val/sum(value),2) for val in value]
beta = round(beta,2)

Portfolio = pd.DataFrame({
    'Stock': stockList,
    'Direction': 'Long',
    'Type': 'S',
    'Stock Price': price,
    'Price': price,
    'Units': units,
    'Value': units*price,
    'Weight': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
Portfolio

NameError: name 'stockList' is not defined

### Weight the Delta’s using Beta

In [ ]:
Portfolio['ASX200 Weighted Delta (point)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']/ASXprices[0]) * Portfolio['Delta'],2) 
Portfolio['ASX200 Weighted Delta (1%)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']) * Portfolio['Delta'] * 0.01,2) 
Portfolio

### Total the Delta’s to get Portfolio Overview

In [ ]:
Portfolio.loc['Total', ['Value', 'ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']] \
= Portfolio[['Value','ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']].sum()
Portfolio

### calculate portfolioPerformance metrics. 

In [ ]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(
            np.dot(weights.T,np.dot(covMatrix, weights))
           )*np.sqrt(252)
    return returns, std

# call it
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)

### Maximium Sharpe Ratio Portfolio

In [ ]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd

def maxSR(meanReturns, covMatrix, riskFreeRate = 0, constraintSet=(0,1)):
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

### Minimium Portfolio Variance

In [ ]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)):
    """Minimize the portfolio variance by altering the 
     weights/allocation of assets in the portfolio"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

### Creating the Efficient Frontier

In [ ]:
def portfolioReturn(weights, meanReturns, covMatrix):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]


def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """For each returnTarget, we want to optimise the portfolio for min variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)

    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

### callable function for graphing package like Plotly/Dash combination

In [ ]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0.03, constraintSet=(0,1)):
    """Read in mean, cov matrix, and other financial information
        Output, Max SR , Min Volatility, efficient frontier """
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList

### Visualising the Efficient Frontier

In [ ]:
def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Return a graph ploting the min vol, max sr and efficient frontier"""
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)

    #Max SR
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_returns],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )

    #Min Vol
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[minVol_std],
        y=[minVol_returns],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )

    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()